## Import Libraries

In [1]:
import retro
import time
import os

from gym import Env
from gym.spaces import MultiBinary, Box
import numpy as np
import cv2
from matplotlib import pyplot as plt

In [2]:
import optuna
from stable_baselines3 import PPO
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.vec_env import DummyVecEnv, VecFrameStack, VecMonitor
from stable_baselines3.common.callbacks import BaseCallback

c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Setting up game

In [3]:
#Search for ROMS
!python -m retro.import .

Importing StreetFighterIISpecialChampionEdition-Genesis
Imported 1 games


In [4]:
#Loads the game env
env = retro.make(game = 'StreetFighterIISpecialChampionEdition-Genesis')

In [8]:
#See the posible imputs that it could have
env.action_space.sample()

array([1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1], dtype=int8)

In [ ]:
#Put game in first state
obs = env.reset()
#Flag
done = False
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        obs, reward, done, info = env.step(env.action_space.sample())
        print(reward)

c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0
0.0


KeyboardInterrupt: 

: 

In [13]:
#Close the env
env.close()

## Preprocess

In [15]:
class StreetFighter(Env):
    def __init__(self):
        super().__init__()
        #Action space and observation space
        self.observation_space = Box(low=0, high=255, shape=(84, 84, 1), dtype=np.uint8)
        self.action_space = MultiBinary(12)
        #Start instance of game
        self.game = retro.make(game = 'StreetFighterIISpecialChampionEdition-Genesis', use_restricted_actions = retro.Actions.FILTERED)
        

    def step(self, action):
        #Take a step
        obs, reward, done, info = self.game.step(action)
        obs = self.preprocess(obs)
        #Frame delta
        frame_delta = obs - self.previous_frame
        self.previous_frame = obs
        #Rewards function
        reward = info['score'] - self.score
        self.score = info['score']
        return frame_delta, reward, done, info


    def render(self, *args, **kwargs):
        self.game.render()
        
        

    def reset(self):
        #Return the first frame
        obs = self.game.reset()
        obs = self.preprocess(obs)
        self.previous_frame = obs
        #Atribute to hold delta
        self.score = 0
        return obs


    def preprocess(self, observation):
        #Grayscaling
        gray = cv2.cvtColor(observation, cv2.COLOR_BGR2GRAY)
        #Resize
        resize = cv2.resize(gray, (84,84), interpolation = cv2.INTER_CUBIC)
        #Add channel so it can be recieved by the model
        channel = np.reshape(resize, (84, 84, 1))
        return channel


    def close(self):
        self.game.close()

In [135]:
env = StreetFighter()

In [20]:
LOG_DIR = './logs/'
OPT_DIR = './opt'

In [34]:
#Return test hyperparameters
def optimize_ppo(trial):
    return {
        'n_steps' : trial.suggest_int('n_steps', 2048, 8192),
        'gamma' : trial.suggest_float('gamma', 0.8, 0.9999),
        'learning_rate' : trial.suggest_float('learning_rate', 1e-5, 1e-4),
        'clip_range' : trial.suggest_float('clip_range', 0.1, 0.4),
        'gae_lambda' : trial.suggest_float('gae_lambda', 0.8, 0.99)
    }

In [94]:
#Run training loop
def optimize_agent(trial):
    try:
        model_params = optimize_ppo(trial)

        env = StreetFighter()
        env = DummyVecEnv([lambda: env])
        env = VecFrameStack(env, 4, channels_order='last')
        env = VecMonitor(env, LOG_DIR)
        
        print(env.observation_space)

        #Create algorithm
        model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=0, **model_params)
        #Can be set higher ex. 100000
        model.learn(total_timesteps=30000)

        #Evaluate model
        mean_reward, _ =evaluate_policy(model, env, n_eval_episodes=5)
        env.close()

        SAVE_PATH = os.path.join(OPT_DIR, 'trial_{}_best_model'.format(trial.number))
        model.save(SAVE_PATH)

        return mean_reward

    except Exception as e:
        print("Error during training:", str(e))
        traceback.print_exc()
        return -10000

In [89]:
env.close()

In [90]:
study = optuna.create_study(direction='maximize')
#n_trials number of tunning of our hyperparameters we pass to our agent, n_jobs number of parallel to train
study.optimize(optimize_agent, n_trials=10, n_jobs=1)

[I 2025-02-09 17:50:05,563] A new study created in memory with name: no-name-98636928-e694-42a1-b5c6-58ab35e73ebf


Box(0, 255, (84, 84, 4), uint8)


c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\stable_baselines3\ppo\ppo.py:149: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2774`, after every 43 untruncated mini-batches, there will be a truncated mini-batch of size 22
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2774 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\stable_baselines3\common\save_util.py:278: UserWarning: Path 'opt' does not exist. Will create it.
  warnings.warn(f"Path '{path.parent}' does not exist. Will create it.")
[I 2025-02-09 17:59:49,171] Trial 0 finished with value: 2400.0 and parameters: {'n_steps': 2774, 'gamma': 0.953912373221467, 'learning_rate': 1.1387497386085902e-05, 'clip_range': 0.2783212648640189, 'gae_lambda': 0.9228443253759401}. Best is trial 0 with value: 2400.0.


Box(0, 255, (84, 84, 4), uint8)


c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\stable_baselines3\ppo\ppo.py:149: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2356`, after every 36 untruncated mini-batches, there will be a truncated mini-batch of size 52
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2356 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2025-02-09 18:10:30,001] Trial 1 finished with value: 4000.0 and parameters: {'n_steps': 2356, 'gamma': 0.9668318127547595, 'learning_rate': 6.264792541868786e-05, 'clip_range': 0.101292935254582, 'gae_lambda': 0.9769169357610252}. Best is trial 1 with value: 4000.0.


...

c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\stable_baselines3\ppo\ppo.py:149: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 2239`, after every 34 untruncated mini-batches, there will be a truncated mini-batch of size 63
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=2239 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2025-02-09 19:19:26,872] Trial 7 finished with value: 4400.0 and parameters: {'n_steps': 2239, 'gamma': 0.8022702008351075, 'learning_rate': 1.8592210944582624e-05, 'clip_range': 0.2379942065243599, 'gae_lambda': 0.8921995264144659}. Best is trial 4 with value: 17400.0.


Box(0, 255, (84, 84, 4), uint8)


c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\stable_baselines3\ppo\ppo.py:149: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3030`, after every 47 untruncated mini-batches, there will be a truncated mini-batch of size 22
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3030 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2025-02-09 19:30:21,995] Trial 8 finished with value: 2900.0 and parameters: {'n_steps': 3030, 'gamma': 0.9320612349373704, 'learning_rate': 3.720704093620379e-05, 'clip_range': 0.1038219037563721, 'gae_lambda': 0.9560920999175759}. Best is trial 4 with value: 17400.0.


Box(0, 255, (84, 84, 4), uint8)


c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\stable_baselines3\ppo\ppo.py:149: UserWarning: You have specified a mini-batch size of 64, but because the `RolloutBuffer` is of size `n_steps * n_envs = 3288`, after every 51 untruncated mini-batches, there will be a truncated mini-batch of size 24
We recommend using a `batch_size` that is a factor of `n_steps * n_envs`.
Info: (n_steps=3288 and n_envs=1)
  f"You have specified a mini-batch size of {batch_size},"
[I 2025-02-09 19:42:18,722] Trial 9 finished with value: 4300.0 and parameters: {'n_steps': 3288, 'gamma': 0.8943881341596287, 'learning_rate': 9.890884959565327e-05, 'clip_range': 0.38241287077271846, 'gae_lambda': 0.8227940848728625}. Best is trial 4 with value: 17400.0.


In [106]:
#Load the best model
model_trained = PPO.load(os.path.join(OPT_DIR, 'trial_4_best_model.zip'))

In [118]:
#Parameters based on the top perfomer, changed the original of 6122 to 6080 by diving by 64
model_params = {
'n_steps': 6080, 
'gamma': 0.8302505769788777, 
'learning_rate': 4.13002243547334e-05, 
'clip_range': 0.28616055774841775, 
'gae_lambda': 0.9400263718743063
}

In [ ]:
class TrainAndLoggingCallback(BaseCallback):

    def __init__(self, check_freq, save_path, verbose=1):
        super(TrainAndLoggingCallback, self).__init__(verbose)
        self.check_freq = check_freq
        self.save_path = save_path

    def _init_callback(self):
        if self.save_path is not None:
            os.makedirs(self.save_path, exist_ok=True)

    def _on_step(self):
        if self.n_calls % self.check_freq == 0:
            model_path = os.path.join(self.save_path, 'best_model_{}'.format(self.n_calls))
            self.model.save(model_path)
        
        return True

In [103]:
CHECKPOINT_DIR = './train/'

In [104]:
callback = TrainAndLoggingCallback(check_freq=10000, save_path=CHECKPOINT_DIR)

## Train model

In [23]:
env = StreetFighter()
env = DummyVecEnv([lambda: env])
env = VecFrameStack(env, 4, channels_order='last')
env = VecMonitor(env, LOG_DIR)

In [119]:
model = PPO('CnnPolicy', env, tensorboard_log=LOG_DIR, verbose=1, **model_params)

Using cpu device
Wrapping the env in a VecTransposeImage.


In [120]:
model.load(os.path.join(OPT_DIR, 'trial_4_best_model.zip'))

In [121]:
model.learn(total_timesteps=30000, callback=callback)

Logging to ./logs/PPO_14
-----------------------------
| time/              |      |
|    fps             | 228  |
|    iterations      | 1    |
|    time_elapsed    | 26   |
|    total_timesteps | 6080 |
-----------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.21e+04    |
|    ep_rew_mean          | 39500.0     |
| time/                   |             |
|    fps                  | 93          |
|    iterations           | 2           |
|    time_elapsed         | 129         |
|    total_timesteps      | 12160       |
| train/                  |             |
|    approx_kl            | 0.015344963 |
|    clip_fraction        | 0.048       |
|    clip_range           | 0.286       |
|    entropy_loss         | -8.31       |
|    explained_variance   | -6.91e-06   |
|    learning_rate        | 4.13e-05    |
|    loss                 | 719         |
|    n_updates            | 10          |
|    poli

## Evaluate model

In [5]:
model = PPO.load('./train/best_model_20000.zip')

In [ ]:
env.close()

: 

In [163]:
mean_reward, _ = evaluate_policy(model, env, render=True, n_eval_episodes=5)

In [164]:
mean_reward

2400.0

In [6]:
#Put game in first state
obs = env.reset()
#Flag
done = False
for game in range(1):
    while not done:
        if done:
            obs = env.reset()
        env.render()
        action = model.predict(obs[0])
        obs, reward, done, info = env.step(action)

c:\Users\saenz\.conda\envs\sf_ai\lib\site-packages\pyglet\image\codecs\wic.py:406: UserWarning: [WinError -2147417850] Cannot change thread mode after it is set
  warnings.warn(str(err))


ValueError: axes don't match array